In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer,RobustScaler
from sklearn.model_selection import GridSearchCV, cross_val_score
from catboost import CatBoostRegressor
import optuna
from catboost import CatBoostRegressor, Pool
from optuna.integration import CatBoostPruningCallback  



In [6]:
base_path = os.path.join("/", "Volumes", "Extreme SSD", "ShellAi")
train_path = os.path.join(base_path, "train.csv")
test_path = os.path.join(base_path, "test.csv")

# Load the CSV files
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [24]:
# Creating  weighted columns
base_features = [col for col in train_df.columns if col not in [f"BlendProperty{i}" for i in range(1, 11)]]
targets = [f"BlendProperty{i}" for i in range(1, 11)]
weighted_data = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_data[weighted_col] = train_df[prop_col] * train_df[frac_col]

# Combine
data_with_weighted = pd.concat([train_df[base_features], pd.DataFrame(weighted_data)], axis=1)
data_with_weighted = data_with_weighted.copy()  # De-fragmented copy
# Add quadratic terms for each component fraction
for i in range(1, 6):
    frac_col = f"Component{i}_fraction"
    quad_col = f"Quadratic_{frac_col}"
    data_with_weighted[quad_col] = train_df[frac_col] ** 2

# Add pairwise products of fractions (all unique pairs to avoid redundancy)
import itertools
fractions = [f"Component{i}_fraction" for i in range(1, 6)]
for pair in itertools.combinations(fractions, 2):
    col_name = f"Pairwise_{pair[0]}_x_{pair[1]}"
    data_with_weighted[col_name] = train_df[pair[0]] * train_df[pair[1]]

# Proceed with scaling (your existing code)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_with_weighted)
scaled_df_features = pd.DataFrame(scaled_features, columns=data_with_weighted.columns)


In [27]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data_with_weighted)
scaled_df_features = pd.DataFrame(scaled_features, columns=data_with_weighted.columns)


pt = PowerTransformer(method='yeo-johnson')
transformed_targets_df = pd.DataFrame(pt.fit_transform(train_df[targets]), columns=targets, index=train_df.index)

# Initial feature selection using Random Forest feature importance
top_features = {}
for target in targets:
    X = scaled_df_features
    y = transformed_targets_df[target]
    rf_initial = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_initial.fit(X, y)
    feature_importance = pd.Series(rf_initial.feature_importances_, index=scaled_df_features.columns).sort_values(ascending=False)
    top_features[target] = feature_importance.head(20).index.tolist()

# GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20,]

}


In [ ]:
warnings.filterwarnings("ignore",category=UserWarning)
for target in targets:
    print(f"\n🔹 Target: {target}")

    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = CatBoostRegressor(verbose=0, random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=5,
                               scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    print(f"Best params: {grid_search.best_params_}")

    # CV MAPE on transformed targets
    cv_mape = -cross_val_score(best_model, X_train, y_train, cv=5,
                               scoring='neg_mean_absolute_percentage_error').mean() * 100
    print(f"Cross-validated MAPE (transformed): {cv_mape:.2f}%")

    # Predict on test set
    y_pred = best_model.predict(X_test)

    # Inverse transform for original scale MAPE
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)

    y_test_inv = pt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[targets.index(target)]

    # Safe MAPE
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    print(f"Test MAPE (original scale): {test_mape:.2f}%")


🔹 Target: BlendProperty1
Best params: {'max_depth': 10, 'n_estimators': 200}


In [ ]:
import optuna
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor

# Suppress warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
best_models = {}  # ← define this before the loop

# Inside the for-loop after training
 # ← define this before the loop

# Inside the for-loop after training


# Loop through each target
for target in targets:
    print(f"\n🔹 Optimizing Target: {target}")

    X = scaled_df_features[top_features[target]]
    y = transformed_targets_df[target]

    # Split for Optuna
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

    # Define objective function
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000),
            "depth": trial.suggest_int("depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
            "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0),
            "loss_function": "MAPE",
            "verbose": 0
        }
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_valid)
        return mean_absolute_percentage_error(y_valid, preds)

    # Run Optuna
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=40, show_progress_bar=True)

    best_params = study.best_params
    print(f"✅ Best params for {target}: {best_params}")

    # Train on full training data
    best_model = CatBoostRegressor(**best_params)
    best_model.fit(X_train_full, y_train_full)

    # Predict on test
    y_pred = best_model.predict(X_test)

    # Inverse transform for original scale MAPE
    y_test_full = pd.DataFrame(np.column_stack([y_test] + [np.zeros(len(y_test))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)
    y_pred_full = pd.DataFrame(np.column_stack([y_pred] + [np.zeros(len(y_pred))]*(len(targets)-1)),
                               columns=targets, index=X_test.index)

    y_test_inv = pt.inverse_transform(y_test_full)[targets.index(target)]
    y_pred_inv = pt.inverse_transform(y_pred_full)[targets.index(target)]

    # Safe MAPE
    mask = y_test_inv != 0
    test_mape = np.mean(np.abs((y_test_inv[mask] - y_pred_inv[mask]) / y_test_inv[mask]) * 100) if mask.any() else 0.0
    best_models[target] = best_model
    print(f"📉 Test MAPE (original scale): {test_mape:.2f}%")



In [ ]:
import numpy as np
import pandas as pd

# STEP 1: Create weighted features for test_df
weighted_data_test = {}
for i in range(1, 6):
    for j in range(1, 11):
        prop_col = f"Component{i}_Property{j}"
        frac_col = f"Component{i}_fraction"
        weighted_col = f"Weighted_Component{i}_Property{j}"
        weighted_data_test[weighted_col] = test_df[prop_col] * test_df[frac_col]

# Add quadratic terms for each component fraction (test data)
for i in range(1, 6):
    frac_col = f"Component{i}_fraction"
    quad_col = f"Quadratic_{frac_col}"
    data_with_weighted_test[quad_col] = test_df[frac_col] ** 2

# Add pairwise products of fractions (test data)
fractions = [f"Component{i}_fraction" for i in range(1, 6)]
for pair in itertools.combinations(fractions, 2):
    col_name = f"Pairwise_{pair[0]}_x_{pair[1]}"
    data_with_weighted_test[col_name] = test_df[pair[0]] * test_df[pair[1]]

# Proceed with scaling (your existing code, using the scaler fitted on training data)
scaled_test_features = scaler.transform(data_with_weighted_test)
scaled_df_test = pd.DataFrame(scaled_test_features, columns=data_with_weighted_test.columns)


# STEP 2: Combine with base features
base_features = [col for col in test_df.columns if col not in [f"BlendProperty{i}" for i in range(1, 11)]]
data_with_weighted_test = pd.concat([test_df[base_features], pd.DataFrame(weighted_data_test)], axis=1)
data_with_weighted_test = data_with_weighted_test.copy()
if 'ID' in data_with_weighted_test.columns:
    data_with_weighted_test = data_with_weighted_test.drop(columns=['ID'])


# STEP 3: Scale test data using training-time scaler (RobustScaler)
scaled_test_features = scaler.transform(data_with_weighted_test)
scaled_df_test = pd.DataFrame(scaled_test_features, columns=data_with_weighted_test.columns)

# STEP 4: Predict for each target using best Optuna model + selected features
all_preds_transformed = []

for target in targets:
    print(f"🔹 Predicting for: {target}")
    
    model = best_models[target]
    selected_features = top_features[target]
    X_test_target = scaled_df_test[selected_features]
    
    y_pred_transformed = model.predict(X_test_target)
    all_preds_transformed.append(y_pred_transformed)

# STEP 5: Inverse transform the predictions to original scale
all_preds_transformed = np.column_stack(all_preds_transformed)  # Shape: (n_samples, 10)
all_preds_original = pt.inverse_transform(all_preds_transformed)

# STEP 6: Create submission DataFrame
submission = pd.DataFrame(all_preds_original, columns=targets, index=test_df.index)

# Include 'ID' column if present
if 'ID' in test_df.columns:
    submission.insert(0, 'ID', test_df['ID'].values)

# STEP 7: Save to CSV
submission.to_csv("submission5.csv", index=False)
print("✅ submission5.csv saved successfully!")